In [1]:
import toolkit

In [2]:
def caseStudyAnalysis(dataSet):
    #print dataSet['times']
    #print dataSet['data'].shape
    #print dataSet['data']['entity'].nunique()
    #print dataSet['data'].columns

    from sklearn.tree import DecisionTreeClassifier
    modelInstance = DecisionTreeClassifier(max_leaf_nodes=8, criterion='entropy')
    modelSimpler = DecisionTreeClassifier(max_leaf_nodes=4, criterion='entropy')
    print "-- Churn model - 2 fold decision tree --"
    churnModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], 2, 'netchurn', modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    folds = 5
    print "-- Churn model - 5 fold decision tree --"
    churnModelMoreFolds = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], folds, 'netchurn', modelInstance, modelSimpler, scoreOnly=False) 

    #print dataSet['data']['netchurn'].mean()
    #print dataSet['data']['netchurn'].var()
    #print dataSet['data']['netchurn'].std()
    #print dataSet['data']['netchurn'].max()
    #print dataSet['data']['netchurn'].min()

    churnBinnedCategories = ['churnLow','churnMedium','churnHigh','churnHigher', 'churnHighest']
    dataSetUpdated = toolkit.utilities.addBinnedResponseCategory(dataSet['data'], 'netchurn', churnBinnedCategories)

    from sklearn.tree import DecisionTreeClassifier
    modelInstance = DecisionTreeClassifier(max_leaf_nodes=8, criterion='entropy')
    print "-- Churn model - 2 fold churnBinnedCategories--"
    churnModelCategories = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, churnBinnedCategories, modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    folds = 3
    print "-- Churn model - 3 fold churnBinnedCategories--"
    churnModelCategories = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, folds, churnBinnedCategories, modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    addedModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSet['data'], 2, 'added', modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    alteredData = dataSet['data'].drop(['netchurn','deleted'], axis=1)
    addedModel = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData, 2, 'added', modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    alteredData2 = dataSet['data'].drop(['netchurn','deleted','n-revs'], axis=1)
    addedModel2 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData2, 2, 'added', modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    alteredData3 = dataSet['data'].drop(['netchurn','deleted','n-revs','n-authors'], axis=1)
    addedModel3 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData3, 2, 'added', modelInstance, modelSimpler, visualize=True, scoreOnly=False) 

    from sklearn.tree import DecisionTreeRegressor
    modelInstanceR = DecisionTreeRegressor(max_leaf_nodes=8)
    modelInstanceRsimpler = DecisionTreeRegressor(max_leaf_nodes=4)    
    alteredData4 = dataSet['data'].drop(['netchurn','deleted','n-revs','n-authors','fractal-value'], axis=1)
    nlineModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData4, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    dataSet['data']['nline'].corr(dataSet['data']['indent_lines'], method='spearman')

    alteredData5 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier'], axis=1)
    nlineModelR2 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData5, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    alteredData6 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier', 'nfunction','nfunction2','nfunction3'], axis=1)
    nlineModelR3 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData6, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    alteredData7 = dataSet['data'].drop(['indent_lines','nchar','nstatement','nidentifier', 'nfunction','nfunction2','nfunction3', 'unique_nidentifier'], axis=1)
    nlineModelR4 = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData7, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    cycloData = dataSet['data'].drop(['cyclomatic_sd', 'cyclomatic_mean'], axis=1)
    cycloModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, cycloData, 2, 'cyclomatic_max', modelInstanceR, visualize=False, scoreOnly=False)

    cycloData2 = dataSet['data'].drop(['halstead_sd','nidentifier','halstead_mean','halstead_min','cyclomatic_sd', 'cyclomatic_mean', 'halstead_max','nstatement','statement_nesting_mean'], axis=1)
    cycloModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, cycloData2, 2, 'cyclomatic_max', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    indentData = dataSet['data'].drop(['indent_sd','indent_median','indent_max','indent_lines'],axis=1)
    indentModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, indentData, 2, 'indent_mean', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

    indentModelC = toolkit.refinement.makeAndUpdateModel(rootDirectory, indentData, 2, 'indent_mean', modelInstance, modelSimpler, visualize=False, scoreOnly=False)

    indentBinnedCategories = ['iLow','iMedium','iHigh','iVeryHigh']
    dataSetUpdated = toolkit.utilities.addBinnedResponseCategory(indentData, 'indent_mean', indentBinnedCategories)
    indentModelC = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, indentBinnedCategories, modelInstance, modelSimpler, visualize=False, scoreOnly=False)

    import psutil
    cores = psutil.cpu_count()

    print "Random forest 2-fold"
    from sklearn.ensemble import RandomForestClassifier
    modelCF = RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=cores)
    modelCFsimpler = RandomForestClassifier(n_estimators=50, max_leaf_nodes=4, criterion='entropy', n_jobs=cores)    
    updatedModelCF = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 2, indentBinnedCategories, modelCF, modelCFsimpler, scoreOnly=False, visualize=False)
    
    print "Random forest 5-fold"
    from sklearn.ensemble import RandomForestClassifier
    modelCF = RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=cores)
    modelCFsimpler = RandomForestClassifier(n_estimators=50, max_leaf_nodes=4, criterion='entropy', n_jobs=cores)    
    updatedModelCF = toolkit.refinement.makeAndUpdateModel(rootDirectory, dataSetUpdated, 5, indentBinnedCategories, modelCF, modelCFsimpler, scoreOnly=False, visualize=False)

In [3]:
def runModels(x,y,folds):
    from sklearn.model_selection import cross_val_score
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.linear_model import SGDClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    
    import psutil
    cores = psutil.cpu_count()
    #folds = 5
    estimators = 100

    '''
    
    print "SVC"
    clf = SVC(kernel='rbf')
    scoresCLF = cross_val_score(clf, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCLF.mean(), scoresCLF.std() * 2))
'''
    print "Naive Bayes"
    gnb = GaussianNB()
    scoresGNB = cross_val_score(gnb, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresGNB.mean(), scoresGNB.std() * 2))

    print "Random Forest"
    modelRF = RandomForestClassifier(n_estimators=estimators, criterion='entropy', n_jobs=cores)
    scoresRF = cross_val_score(modelRF, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresRF.mean(), scoresRF.std() * 2))

    print "Decision Tree"
    dt = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
    scoresDT = cross_val_score(dt, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresDT.mean(), scoresDT.std() * 2))
    
    print "Extremely Random Tree"
    edt = ExtraTreesClassifier(n_estimators=estimators, max_depth=None, min_samples_split=2, random_state=0)
    scoresEDT = cross_val_score(edt, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresEDT.mean(), scoresEDT.std() * 2))
    '''
    print "MLP"
    mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(61, 30, 20, 15, 5, 1), random_state=1)
    scoresMLP = cross_val_score(mlp, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresMLP.mean(), scoresMLP.std() * 2))

    print "AdaBoost"
    ada = AdaBoostClassifier(n_estimators=estimators)
    scoresADA = cross_val_score(ada, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresADA.mean(), scoresADA.std() * 2))

    print "SGD"
    sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=20)
    scoresSGD = cross_val_score(sgd, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresSGD.mean(), scoresSGD.std() * 2)) 
    
    print "Gradient Boost"
    gb = GradientBoostingClassifier(n_estimators=estimators, learning_rate=1.0, max_depth=1, random_state=0)
    scoresGB = cross_val_score(gb, x, y, cv=folds)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scoresGB.mean(), scoresGB.std() * 2))
'''


In [5]:
rootDirectory = '../dataSets/vimStudy/'
metricsDataVim = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/vim/vim', rootDirectory+'vim/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
#caseStudyAnalysis(metricsDataVim)

ValueError: No objects to concatenate

In [ ]:
import numpy as np
x = metricsDataVim['data'].drop('netchurn',axis=1).drop('time', axis=1).drop('entity',axis=1)
y = np.array(metricsDataVim['data']['added'])
runModels(x,y,5)

In [ ]:
import numpy as np
#x = metricsDataVim['data'].drop('netchurn',axis=1).drop('time', axis=1).drop('entity',axis=1)
x = metricsDataVim['data'].drop('netchurn', axis=1).drop('time', axis=1).drop('entity',axis=1)
#x = metricsDataVim['data'].drop(['entity','time','indent_lines','nchar','nstatement','nidentifier'], axis=1)
y = metricsDataVim['data']['netchurn']

#runModels(x,y,10)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
x2 = metricsDataVim['data'].drop(['netchurn','deleted','n-revs','n-authors','fractal-value'], axis=1).drop('time', axis=1).drop('entity',axis=1)

print "MLP"
mlp = MLPClassifier(solver='adam', alpha=1e-8, hidden_layer_sizes=(512,256,128), random_state=1)
scoresMLP = cross_val_score(mlp, x2, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresMLP.mean(), scoresMLP.std() * 2))
    
#nlineModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData4, 2, 'nline', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)


In [ ]:
rootDirectory = '../dataSets/openSSLstudy/'
metricsDataOpenSSL = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/openssl/openssl', rootDirectory+'openssl/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
#caseStudyAnalysis(metricsDataOpenSSL)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
modelInstanceR = DecisionTreeRegressor(max_leaf_nodes=8)
modelInstanceRsimpler = DecisionTreeRegressor(max_leaf_nodes=4)
alteredData4 = metricsDataOpenSSL['data']
nlineModelR = toolkit.refinement.makeAndUpdateModel(rootDirectory, alteredData4, 3, 'cbo', modelInstanceR, modelInstanceRsimpler, visualize=False, scoreOnly=False)

In [ ]:
import numpy as np
x = metricsDataOpenSSL['data'].drop('netchurn',axis=1).drop('time', axis=1).drop('entity',axis=1)
y = np.array(metricsDataOpenSSL['data']['added'])
runModels(x,y,10)

In [ ]:
rootDirectory = '../dataSets/httpdStudy/'
metricsDataHTTPD = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/apache/httpd', rootDirectory+'httpd/', '*/*.c */*.h *.c *.h', ['indent','c'], branch='trunk', skipEvery=50)
#caseStudyAnalysis(metricsDataHTTPD)

In [ ]:
import numpy as np
x = metricsDataHTTPD['data'].drop('netchurn',axis=1).drop('time', axis=1).drop('entity',axis=1)
y = np.array(metricsDataHTTPD['data']['added'])
runModels(x,y,10)

In [ ]:
rootDirectory = '../dataSets/nginxStudy/'
metricsDataNginx = toolkit.data.gatherTimeMetrics(rootDirectory, 'https://github.com/nginx/nginx', rootDirectory+'nginx/', '*/*.c */*.h *.c *.h', ['indent','c'], skipEvery=50)
#caseStudyAnalysis(metricsDataNginx)

In [ ]:
import numpy as np
x = metricsDataNginx['data'].drop('netchurn',axis=1).drop('time', axis=1).drop('entity',axis=1)
y = np.array(metricsDataNginx['data']['added'])
runModels(x,y,10)

# 